In [1]:
from scipy.sparse import csr_matrix, coo_matrix
from symmer.chem import MoleculeBuilder
from symmer.symplectic import random_PauliwordOp, symplectic_to_sparse_matrix
from symmer.projection import QubitTapering, CS_VQE, StabilizerIdentification
from symmer.symplectic import AnsatzOp, ObservableOp
from symmer.utils import exact_gs_energy
import multiprocessing as mp
import numpy as np
import time
import json

print(f'No logical cores: {mp.cpu_count()}')

with open('notebooks/data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)

No logical cores: 384


In [2]:
speciesname = 'HCN_STO-3G_SINGLET'
# build the molecule
mol_data = molecule_geometries[speciesname]
atoms  = mol_data['atoms']
coords = mol_data['coords']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=0, basis='STO-3G', spin=0, run_fci=True, print_info=True)

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# initiate stabilizer identification classes
CC_stabilizers = StabilizerIdentification(ucc_tap)
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')

Molecule geometry:
C	0.0	0.0	-0.500078
H	0.0	0.0	-1.56999
N	0.0	0.0	0.652923

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -91.67520896743608
MP2 energy:  -91.82881563197405
CCSD energy: -91.83755425388145
FCI energy:  -91.84091227094369


Number of qubits: 22


In [3]:
exact_gs_energy(ham_tap.to_sparse_matrix)[0]

KeyboardInterrupt: 

In [7]:
import qiskit
from qiskit.quantum_info import Pauli

In [18]:
Pauli([1,1],[0,1]).to_operator()

Operator([[0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j],
          [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))

In [24]:
from openfermion import get_sparse_operator

In [25]:
get_sparse_operator(ham_tap.to_QubitOperator)

KeyboardInterrupt: 

In [3]:
from functools import reduce

def convert_to_sparse_parallel(self):
    # initiate a pool for parallel processing of Pauli multiplication
    pool = mp.Pool(mp.cpu_count()//2)#, maxtasksperchild=10)
    matrix_elements = []
    # loop over the Pauli terms of right-hand PauliwordOp and parallelize
    for Pvec_single, coeff_single in zip(self.symp_matrix, self.coeff_vec):
        pool.apply_async(
            symplectic_to_sparse_matrix, 
            args=(Pvec_single, coeff_single), 
            callback=lambda x:matrix_elements.append(x)
        )
    # close the pool and let all the processes complete    
    pool.close()
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
    print('reduce phase')
    
    pool = mp.Pool(mp.cpu_count())#, maxtasksperchild=10)
    
    pool.close()
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
    print('reduce phase')
    
    return reduce(lambda x,y:x+y, matrix_elements)

In [4]:
self = ham_tap
pool = mp.Pool(mp.cpu_count()//2)#, maxtasksperchild=10)
matrix_elements = []
# loop over the Pauli terms of right-hand PauliwordOp and parallelize
for Pvec_single, coeff_single in zip(self.symp_matrix, self.coeff_vec):
    pool.apply_async(
        symplectic_to_sparse_matrix, 
        args=(Pvec_single, coeff_single), 
        callback=lambda x:matrix_elements.append(x)
    )
# close the pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

In [5]:
len(matrix_elements)/17

405.4117647058824

In [6]:
#from itertools import izip_longest as zip_longest # for Python 2.x
from itertools import zip_longest # for Python 3.x
#from six.moves import zip_longest # for both (uses the six compat library)

def grouper(n, iterable, padvalue=None):
    "grouper(3, 'abcdefg', 'x') --> ('a','b','c'), ('d','e','f'), ('g','x','x')"
    return zip_longest(*[iter(iterable)]*n, fillvalue=padvalue)

In [ ]:
pool = mp.Pool(mp.cpu_count()//2)#, maxtasksperchild=10)

def reduce_sum(List):
    reduce(lambda x,y:x+y, List)

reduced_sum = []
sum_groups = grouper(len(matrix_elements)//(mp.cpu_count()//2), matrix_elements)
# loop over the Pauli terms of right-hand PauliwordOp and parallelize
for to_sum in sum_groups:
    pool.apply_async(
        reduce_sum, 
        args=(to_sum), 
        callback=lambda x:reduced_sum.append(x)
    )
# close the pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

Process ForkPoolWorker-193:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-194:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in ge

AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-208:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-209:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()


  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-223:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-224:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._ar

  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-238:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-239:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "

  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-253:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-254:
Traceback (most rece

...

  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-328:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-329:
Traceback (most rece

AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
Process ForkPoolWorker-343:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler

AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-357:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-358:
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/p

  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-372:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'reduce_sum' on <module '__main__'>
Process ForkPoolWorker-373:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._ar

Process ForkPoolWorker-507:
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-492:
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-541:
Process ForkPoolWorker-567:
Process ForkPoolWorker-566:
Process ForkPoolWorker-551:
Process ForkPoolWorker-

In [13]:
S = CC_stabilizers.symmetry_basis_by_subspace_dimension(6)
H_cs = cs_vqe.project_onto_subspace(S)
CC_cs = cs_vqe.project_onto_subspace(S, aux_operator=ucc_tap)
exact_gs_energy(H_cs.to_sparse_matrix)[0] - molecule.fci_energy

0.05472776074462615

In [5]:
CC_truncated = CC_cs.sort()

obs = ObservableOp(H_cs.symp_matrix, H_cs.coeff_vec)
anz = AnsatzOp(CC_truncated.symp_matrix, CC_truncated.coeff_vec)
ref_state = hf_tapered[cs_vqe.free_qubit_indices]

In [6]:
obs.evaluation_method = 'trotter_rotations'

In [7]:
#obs.VQE(ansatz_op=anz, ref_state=ref_state)[0]['fun'] - molecule.fci_energy

In [8]:
H_cs.to_sparse_matrix

<64x64 sparse matrix of type '<class 'numpy.complex128'>'
	with 576 stored elements in Compressed Sparse Row format>

In [14]:
from functools import reduce

def convert_to_sparse_parallel(self):
    # initiate a pool for parallel processing of Pauli multiplication
    pool = mp.Pool(mp.cpu_count())#, maxtasksperchild=10)
    matrix_elements = []
    # loop over the Pauli terms of right-hand PauliwordOp and parallelize
    for Pvec_single, coeff_single in zip(self.symp_matrix, self.coeff_vec):
        pool.apply_async(
            symplectic_to_sparse_matrix, 
            args=(Pvec_single, coeff_single), 
            callback=lambda x:matrix_elements.append(x)
        )
    # close the pool and let all the processes complete    
    pool.close()
    pool.join()  # postpones the execution of next line of code until all processes in the queue are done.
    print('reduce phase')
    return reduce(lambda x,y:x+y, matrix_elements)

In [ ]:
convert_to_sparse_parallel(ham_tap)

Process ForkPoolWorker-1395:
Process ForkPoolWorker-938:
Process ForkPoolWorker-1044:
Process ForkPoolWorker-1343:
Process ForkPoolWorker-1398:
Process ForkPoolWorker-1360:
Process ForkPoolWorker-1299:
Process ForkPoolWorker-1336:
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/threading.py", line 919, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.6/threading.py", line 867, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 463, in _handle_results
    task = get()
  File "/usr/lib64/python3.6/multiprocessing/connection.py", line 255, in recv
    return _ForkingPickler.loads(buf.getbuffer())
_pickle.UnpicklingError: invalid load key, '4'.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Process ForkPoolWorker-1369:
Process ForkPoolWorker-1393:
Process ForkPoolWorker-1379:
Process ForkPoolWorker-1416:
ERROR:

In [ ]:
ham_tap.to_sparse_matrix